# 09.03.01 - Classification

## Purpose

This notebook will go over two classification strategies using the test set we talked about in 09.01 and 09.02

## Libraries

* Sklearn

## References/Reading

In [1]:
import pandas as pd
from seaborn import load_dataset
from sklearn.model_selection import train_test_split

In [2]:
# Helper methods
def createCategoricalDummies(dataFrame, categoryList):
    return pd.get_dummies(dataFrame[categoryList], prefix_sep = "::", drop_first = True)

# Part 1: Prepare the data
We've seen all this already, we'll just do it in one spot

In [3]:
titanicDataSet = load_dataset("titanic")
columns = ["survived", "pclass", "sex", "age", "sibsp", "parch", "fare", "embarked"]
categories = ["embarked", "sex"]

In [4]:
titanicDataSet = titanicDataSet[columns]
titanicDataSet.dropna(inplace=True)
titanicDataSet = pd.concat(
    [titanicDataSet.drop(categories, axis=1), createCategoricalDummies(titanicDataSet, categories)], axis= 1)

features = list(titanicDataSet.columns)
features.remove("survived")
target = "survived"

In [5]:
X = titanicDataSet[features]
y = titanicDataSet[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)